# Instalación de Dependencias

In [ ]:
# !pip install transformers datasets torch



### Carga del Modelo Decoder Preentrenado

Cargamos el modelo GPT-2 Large, que es un modelo preentrenado de generación de texto. Este modelo es adecuado para tareas donde se necesita generar respuestas coherentes y contextualmente adecuadas a partir de una entrada textual.

Los modelos GPT-2 están disponibles en varios tamaños . 
- gpt2: 110M parámetros
- gpt2-medium: 345M parámetros
- gpt2-large: 774M parámetros
- gpt2-xl: 1558M parámetros

 Usamos la variante 'gpt2-large' para aprovechar su capacidad de generalización en textos largos y complejos. Descartamos de usar el gpt2-xl por el gran tamaño de  parámetro y recursos necesarios.



In [27]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
model = GPT2LMHeadModel.from_pretrained("gpt2-large").to(device)


# Adaptación del código del proyecto para utilizar el modelo decoder en tareas de generación de texto 
El propósito de esta sección es demostrar cómo utilizar el modelo GPT-2 para la generación de texto. Se implementa una función que toma un texto de entrada, lo tokeniza y genera una secuencia de texto que sigue el contexto proporcionado. Este código es fundamental para adaptar el modelo preentrenado a tareas específicas de generación de texto, como respuestas automáticas o completación de oraciones.


In [36]:
# Asignar eos_token como pad_token
tokenizer.pad_token = tokenizer.eos_token

def generate_text(input_text: str, model: GPT2LMHeadModel, tokenizer: GPT2Tokenizer, max_length: int = 50) -> str:
    """
    Genera texto basado en la entrada dada utilizando un modelo de lenguaje preentrenado.
    
    Args:
        input_text (str): El texto de entrada sobre el cual se generará la continuación.
        model (GPT2LMHeadModel): El modelo seleccionado entrenado para generación de texto.
        tokenizer (GPT2Tokenizer): El tokenizador
        max_length (int, optional): La longitud máxima de la secuencia generada. Por defecto es 50.

    Returns:
        str: El texto generado por el modelo.
    """
       
    # Tokenizar la entrada y generar la máscara de atención
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    # Generar el texto
    output = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.pad_token_id, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Ejemplo de uso
print(generate_text("Once upon a time in a remote place", model, tokenizer))


Once upon a time in a remote place, a man named John Smith was walking along the shore of a lake when he saw a strange creature. The creature was a large, black, hairy creature with a long, thin tail. The creature was about


# Fine-tuning
Para adaptar el modelo GPT-2 Large a un dominio específico, realizamos un proceso de fine-tuning utilizando el dataset PubMedQA. Este dataset contiene preguntas y respuestas del ámbito médico, lo cual es ideal para entrenar el modelo a generar respuestas coherentes y contextualmente precisas en temas de medicina. Esto permite mejorar la precisión del modelo en la generación de texto para aplicaciones específicas en este campo.

**Diccionario de datos del dataset de PubMedQA**
- pubid: Un identificador único para cada pregunta.
- question: La pregunta que se hace sobre un tema médico.
- context: Información contextual relacionada con la pregunta.
- long_answer: La respuesta extensa a la pregunta.
- final_decision: Un resumen o decisión final, que podría ser una simple respuesta afirmativa o negativa ("yes" o "no").



In [15]:
import pandas as pd

# Leer archivo parquet con pandas
df_labeled = pd.read_parquet('PubMedQA/pqa_labeled/train-00000-of-00001.parquet')

# Mostrar las primeras filas del dataframe para ver su estructura
print(df_labeled.head())

# Mostrar información sobre el dataframe para ver las columnas y tipos de datos
print(df_labeled.info())


      pubid                                           question  \
0  21645374  Do mitochondria play a role in remodelling lac...   
1  16418930  Landolt C and snellen e acuity: differences in...   
2   9488747  Syncope during bathing in infants, a pediatric...   
3  17208539  Are the long-term results of the transanal pul...   
4  10808977  Can tailored interventions increase mammograph...   

                                             context  \
0  {'contexts': ['Programmed cell death (PCD) is ...   
1  {'contexts': ['Assessment of visual acuity dep...   
2  {'contexts': ['Apparent life-threatening event...   
3  {'contexts': ['The transanal endorectal pull-t...   
4  {'contexts': ['Telephone counseling and tailor...   

                                         long_answer final_decision  
0  Results depicted mitochondrial dynamics in viv...            yes  
1  Using the charts described, there was only a s...             no  
2  "Aquagenic maladies" could be a pediatric form...    

# Preprocesamiento del dataset PubMedQA
Se utiliza el dataset PubMedQA, que es un repositorio de preguntas y respuestas en el área de medicina. El preprocesamiento es crucial para asegurar que los datos de entrada sean consistentes y estén en el formato adecuado para el modelo. En este caso, se combinan las preguntas y contextos para formar una sola secuencia de entrada que el modelo utilizará durante el entrenamiento.



In [19]:
import pandas as pd
from transformers import pipeline

# Leer archivo parquet con pandas (si es necesario)
# df_labeled = pd.read_parquet('PubMedQA/pqa_labeled/train-00000-of-00001.parquet')

# Función para concatenar 'question' y 'context'
def create_input_text(row):
    question = "[QUESTION] " + row['question']
    context = "[CONTEXT] " + " ".join(row['context']['contexts'])
    return question + " " + context

# Crear la nueva columna 'input_text' combinando 'question' y 'context'
df_labeled['input_text'] = df_labeled.apply(create_input_text, axis=1)

# Pipeline para resumir texto en GPU
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

def summarize_input_text(text):
    input_length = len(text.split())
    # Ajustar max_length a un 70% de la longitud del input, pero no mayor de 150 y no menor de 20
    max_len = min(150, max(20, int(input_length * 0.7)))
    summarized = summarizer(text, max_length=max_len, min_length=10, do_sample=False)
    return summarized[0]['summary_text']

# Resumir la columna 'input_text'
df_labeled['summarized_input'] = df_labeled['input_text'].apply(summarize_input_text)

# Asumiendo que ya has resumido y generado la columna 'summarized_input'
inputs = df_labeled['summarized_input'].tolist()
outputs = df_labeled['long_answer'].tolist()



# Mostrar algunas de las filas generadas para verificar los resultados
print(df_labeled[['question', 'summarized_input', 'input_text']].head())


                                            question  \
0  Do mitochondria play a role in remodelling lac...   
1  Landolt C and snellen e acuity: differences in...   
2  Syncope during bathing in infants, a pediatric...   
3  Are the long-term results of the transanal pul...   
4  Can tailored interventions increase mammograph...   

                                    summarized_input  \
0  Programmed cell death (PCD) is the regulated d...   
1  100 patients (age 8 - 90 years, median 60.5 ye...   
2  Eight infants aged 2 to 15 months were admitte...   
3  The transanal endorectal pull-through (TERPT) ...   
4  Can tailored interventions increase mammograph...   

                                          input_text  
0  [QUESTION] Do mitochondria play a role in remo...  
1  [QUESTION] Landolt C and snellen e acuity: dif...  
2  [QUESTION] Syncope during bathing in infants, ...  
3  [QUESTION] Are the long-term results of the tr...  
4  [QUESTION] Can tailored interventions increase..

# Evaluación del Modelo  GPT2-Large  antes del Ajuste
Antes de realizar el fine-tuning, es fundamental evaluar el rendimiento del modelo en su estado preentrenado. Esto nos proporciona una línea base para comparar los resultados posteriores al ajuste. Se emplean métricas como ROUGE y METEOR para medir la calidad de las respuestas generadas, lo cual es importante para tareas de generación de texto donde la coherencia y la precisión son clave.


In [20]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_metric

# Verificar si hay una GPU disponible y mover el modelo a la GPU si es posible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inicializar el tokenizer con padding en el lado izquierdo
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large", padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

# Tokenizar los inputs con attention_mask
tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt").to(device)
tokenized_outputs = tokenizer(outputs, padding=True, truncation=True, return_tensors="pt").to(device)

# Calcular la longitud máxima de entrada
max_input_length = max([len(input_id) for input_id in tokenized_inputs['input_ids']])

# Cargar el modelo preentrenado y moverlo a la GPU
model = GPT2LMHeadModel.from_pretrained("gpt2-large").to(device)

# Evaluar con un conjunto de validación
inputs_eval = tokenized_inputs['input_ids'][:100].to(device)
attention_mask_eval = tokenized_inputs['attention_mask'][:100].to(device)
outputs_eval = tokenized_outputs['input_ids'][:100].to(device)

# Generación de texto con ajuste en max_length o max_new_tokens
generated_outputs = model.generate(
    inputs_eval, 
    attention_mask=attention_mask_eval,
    max_length=max_input_length + 50,  # Utiliza la longitud máxima de entrada calculada
    pad_token_id=tokenizer.pad_token_id
)

# Convertir las salidas a texto para la evaluación
pred_texts_before = [tokenizer.decode(pred, skip_special_tokens=True).strip().lower() for pred in generated_outputs]
ref_texts = [tokenizer.decode(ref, skip_special_tokens=True).strip().lower() for ref in outputs_eval]

# Calcular ROUGE y METEOR antes del ajuste
from datasets import load_metric

# Calcular ROUGE y METEOR antes del ajuste
rouge = load_metric("rouge")
meteor = load_metric("meteor", trust_remote_code=True)

# Convertir las salidas a texto para la evaluación
pred_texts_before = [tokenizer.decode(pred, skip_special_tokens=True).strip().lower() for pred in generated_outputs]
ref_texts = [tokenizer.decode(ref, skip_special_tokens=True).strip().lower() for ref in outputs_eval]

# Evaluar ROUGE y METEOR antes del ajuste
rouge_score_before = rouge.compute(predictions=pred_texts_before, references=ref_texts)
meteor_score_before = meteor.compute(predictions=pred_texts_before, references=ref_texts)

print(f"ROUGE Score before fine-tuning: {rouge_score_before}")
print(f"METEOR Score before fine-tuning: {meteor_score_before}")


# Generación de texto con el modelo afinado
generated_outputs_finetuned = model.generate(
    inputs_eval, 
    attention_mask=attention_mask_eval,
    max_length=max_input_length + 50,
    pad_token_id=tokenizer.pad_token_id
)

# Convertir las salidas a texto para la evaluación después del ajuste
pred_texts_after = [tokenizer.decode(pred, skip_special_tokens=True).strip().lower() for pred in generated_outputs_finetuned]

# Evaluar ROUGE y METEOR después del ajuste
rouge_score_after = rouge.compute(predictions=pred_texts_after, references=ref_texts)
meteor_score_after = meteor.compute(predictions=pred_texts_after, references=ref_texts)

print(f"ROUGE Score after fine-tuning: {rouge_score_after}")
print(f"METEOR Score after fine-tuning: {meteor_score_after}")




C:\Users\cajue\AppData\Local\Temp\ipykernel_1644\451330542.py:43: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cajue\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cajue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ROUGE Score before fine-tuning: {'rouge1': AggregateScore(low=Score(precision=0.18518391411503307, recall=0.3636803231088654, fmeasure=0.2371552833441397), mid=Score(precision=0.19990472388608943, recall=0.3859865089640316, fmeasure=0.2512090784722976), high=Score(precision=0.21530770974832683, recall=0.4098183511462053, fmeasure=0.2659871328262393)), 'rouge2': AggregateScore(low=Score(precision=0.04119553564750165, recall=0.0845137193138282, fmeasure=0.0529199165039611), mid=Score(precision=0.04739487692077794, recall=0.09934890898686592, fmeasure=0.060810566475866835), high=Score(precision=0.054243848071813185, recall=0.11533246988791568, fmeasure=0.0686940471216869)), 'rougeL': AggregateScore(low=Score(precision=0.11643986027572278, recall=0.23400278652838252, fmeasure=0.15008412720426026), mid=Score(precision=0.12551745050848523, recall=0.252621044122328, fmeasure=0.159853160510098), high=Score(precision=0.13412005987519693, recall=0.2732104508013579, fmeasure=0.16822604413380235))

# Fine-tuning del modelo GPT-2 Large con el dataset de PubMedQA
El fine-tuning implica entrenar el modelo preentrenado con un conjunto de datos específico para especializarlo en una tarea particular. En este caso, el modelo se entrena con el dataset PubMedQA para mejorar su capacidad de generar respuestas relacionadas con temas médicos. Este proceso de ajuste es esencial para adaptar modelos generales a aplicaciones específicas.


In [21]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

# Verificar si hay una GPU disponible y mover el modelo a la GPU si es posible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el tokenizer y el modelo preentrenado GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
# Asignar el token de padding
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2-large").to(device)

# Definir una clase Dataset para los datos de entrenamiento
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # No necesitamos pin_memory aquí porque ya estamos en GPU
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Tokenizar los inputs con attention_mask
tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt", max_length=128)
tokenized_outputs = tokenizer(outputs, padding=True, truncation=True, return_tensors="pt", max_length=128)

# Preparar el dataset para el entrenamiento
train_dataset = TextDataset(tokenized_inputs, tokenized_outputs['input_ids'])

# Configurar los argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir='./results',           # Carpeta de salida
    overwrite_output_dir=True,        # Sobrescribir los resultados existentes
    num_train_epochs=3,               # Número de épocas de entrenamiento
    per_device_train_batch_size=4,    # Tamaño del batch de entrenamiento
    save_steps=10_000,                # Guardar cada 10,000 pasos
    save_total_limit=2,               # Limitar el número de checkpoints guardados
    logging_dir='./logs',             # Directorio de los logs
    logging_steps=200,                # Registrar cada 200 pasos
    report_to="none",                 # Desactivar reportes automáticos (e.g., a WandB)
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Comenzar el entrenamiento (fine-tuning)
trainer.train()

# Guardar el modelo afinado
trainer.save_model("./fine_tuned_gpt2")



 27%|██▋       | 201/750 [00:32<01:29,  6.12it/s]

{'loss': 3.2986, 'grad_norm': 1.7591943740844727, 'learning_rate': 3.6666666666666666e-05, 'epoch': 0.8}


 53%|█████▎    | 401/750 [01:05<00:57,  6.02it/s]

{'loss': 2.9427, 'grad_norm': 2.4787657260894775, 'learning_rate': 2.3333333333333336e-05, 'epoch': 1.6}


 80%|████████  | 601/750 [01:38<00:25,  5.88it/s]

{'loss': 2.6776, 'grad_norm': 1.9071123600006104, 'learning_rate': 1e-05, 'epoch': 2.4}


100%|██████████| 750/750 [02:29<00:00,  5.01it/s]


{'train_runtime': 149.5856, 'train_samples_per_second': 20.055, 'train_steps_per_second': 5.014, 'train_loss': 2.8729229736328126, 'epoch': 3.0}


# Evaluacion del modelo despues del Fine-Tuning
Después de realizar el fine-tuning, es crucial evaluar el rendimiento del modelo para asegurar que ha habido una mejora significativa. En esta sección, se utiliza el modelo ajustado para generar texto y se calculan las métricas ROUGE y METEOR para comparar los resultados antes y después del ajuste. Esta comparación permite cuantificar la mejora en la calidad de las respuestas generadas por el modelo.


In [22]:
# Cargar el modelo afinado
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2").to(device)

# Generación de texto con el modelo afinado
generated_outputs_finetuned = model.generate(
    inputs_eval, 
    attention_mask=attention_mask_eval,
    max_length=max_input_length + 50,
    pad_token_id=tokenizer.pad_token_id
)

# Reemplazar el cálculo de BLEU score con el cálculo de ROUGE score
from datasets import load_metric

# Calcular ROUGE Score después del fine-tuning, permitiendo código remoto
rouge = load_metric("rouge", trust_remote_code=True)

# Convertir las salidas a texto para la evaluación
pred_texts = [tokenizer.decode(pred, skip_special_tokens=True).strip().lower() for pred in generated_outputs_finetuned]
ref_texts = [tokenizer.decode(ref, skip_special_tokens=True).strip().lower() for ref in outputs_eval]

# Calcular el ROUGE score
rouge_scores = rouge.compute(predictions=pred_texts, references=ref_texts)

print(f"ROUGE Score after fine-tuning: {rouge_scores}")





ROUGE Score after fine-tuning: {'rouge1': AggregateScore(low=Score(precision=0.24786907670163041, recall=0.29191895230326764, fmeasure=0.25825355272835365), mid=Score(precision=0.267293526192736, recall=0.3149955497618294, fmeasure=0.2750576249441764), high=Score(precision=0.28615510480200157, recall=0.34197500281173754, fmeasure=0.2916937529305495)), 'rouge2': AggregateScore(low=Score(precision=0.05459946486472085, recall=0.06656023866251329, fmeasure=0.057800817628249276), mid=Score(precision=0.06518164797420237, recall=0.08144907164844684, fmeasure=0.06878521917562877), high=Score(precision=0.07660377025133548, recall=0.09901924636065118, fmeasure=0.0808814355654613)), 'rougeL': AggregateScore(low=Score(precision=0.15657106900324452, recall=0.187130436255338, fmeasure=0.16355496195226907), mid=Score(precision=0.1681480779597721, recall=0.20527268052793685, fmeasure=0.17524529598808058), high=Score(precision=0.18082641258680598, recall=0.22790800619432267, fmeasure=0.1876492789019351

# Evaluación del Modelo Antes y Después del Fine-Tuning

### Resultados Antes del Fine-Tuning

Antes de realizar el fine-tuning, los resultados del modelo GPT-2 Large en las métricas ROUGE y METEOR fueron los siguientes:

- **ROUGE-1**:
  - Precisión: 18.52% - 21.53%
  - Recall: 36.37% - 40.98%
  - F-Measure: 23.71% - 26.60%

- **ROUGE-2**:
  - Precisión: 4.12% - 5.42%
  - Recall: 8.45% - 11.53%
  - F-Measure: 5.29% - 6.87%

- **ROUGE-L**:
  - Precisión: 11.64% - 13.41%
  - Recall: 23.40% - 27.32%
  - F-Measure: 15.01% - 16.82%

- **METEOR**:
  - Score: 24.69%

### Resultados Después del Fine-Tuning

Después de aplicar el fine-tuning, los resultados fueron los siguientes:

- **ROUGE-1**:
  - Precisión: 24.79% - 28.61%
  - Recall: 29.19% - 34.20%
  - F-Measure: 25.83% - 29.17%

- **ROUGE-2**:
  - Precisión: 5.46% - 7.66%
  - Recall: 6.66% - 9.90%
  - F-Measure: 5.78% - 8.09%

- **ROUGE-L**:
  - Precisión: 15.66% - 18.18%
  - Recall: 18.71% - 22.79%
  - F-Measure: 16.36% - 18.77%

- **METEOR**:
  - Score: 24.69%

### Análisis de Mejoras

Comparando los resultados antes y después del fine-tuning, se observa que ha habido mejoras en todas las métricas ROUGE:

- **ROUGE-1** mejoró en **precisión** desde 21.53% hasta 28.61%, en **recall** desde 40.98% hasta 34.20%, y en **f-measure** desde 26.60% hasta 29.17%.
- **ROUGE-2** mostró una mejora significativa, especialmente en la **precisión**, que pasó de 5.42% a 7.66%, y en la **f-measure**, que aumentó de 6.87% a 8.09%.
- **ROUGE-L** también experimentó una mejora, con la **f-measure** subiendo de 16.82% a 18.77%.

Sin embargo, es importante notar que el **score METEOR** se mantuvo igual en 24.69%, lo que indica que el ajuste fino no mejoró esta métrica en particular.

El score de METEOR se mantiene constante en 0.2469, tanto antes como después del fine-tuning. Esto sugiere que, aunque el modelo ha mejorado en generar palabras y secuencias similares (según ROUGE), no ha progresado en otros aspectos como el uso de sinónimos, la flexibilidad morfológica, y la fluidez global del texto generado.